<a href="https://colab.research.google.com/github/alxmzr/Colab/blob/main/Gann_Astro_Intraday_Analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install skyfield ephem pytz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.0/367.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.5/234.5 kB 24.2 MB/s eta 0:00:00


In [4]:
from skyfield.api import load
from skyfield.api import N, E, wgs84
from skyfield.almanac import find_discrete, moon_phases
import ephem
from datetime import datetime, timedelta
import pytz

# Указанная дата анализа
DATE = '2025-07-08'
UTC = pytz.UTC
start_time = UTC.localize(datetime.strptime(DATE, "%Y-%m-%d"))
end_time = start_time + timedelta(days=1)

# Загрузка эфемерид
eph = load('de421.bsp')
ts = load.timescale()

# Время начала и конца дня
t0 = ts.utc(start_time.year, start_time.month, start_time.day)
t1 = ts.utc(end_time.year, end_time.month, end_time.day)

# Планеты
planets = {
    'Sun': eph['sun'],
    'Moon': eph['moon'],
    'Mercury': eph['mercury'],
    'Venus': eph['venus'],
    'Mars': eph['mars'],
    'Jupiter': eph['jupiter barycenter'],
    'Saturn': eph['saturn barycenter'],
    'Uranus': eph['uranus barycenter'],
    'Neptune': eph['neptune barycenter'],
    'Pluto': eph['pluto barycenter'],
}

aspect_angles = {
    0: 'Соединение',
    60: 'Секстиль',
    90: 'Квадрат',
    120: 'Тригон',
    180: 'Оппозиция',
}

def calc_angle(p1, p2, time):
    e1 = planets[p1].at(time).ecliptic_latlon()[1].degrees
    e2 = planets[p2].at(time).ecliptic_latlon()[1].degrees
    diff = abs(e1 - e2) % 360
    return min(diff, 360 - diff)

def find_aspects():
    times = []
    for hour in range(0, 24):
        t = ts.utc(start_time.year, start_time.month, start_time.day, hour)
        for p1 in planets:
            for p2 in planets:
                if p1 == p2:
                    continue
                angle = calc_angle(p1, p2, t)
                for target_angle, name in aspect_angles.items():
                    if abs(angle - target_angle) < 2.0:  # допуск 2°
                        times.append((t.utc_datetime(), f'{p1} {name} {p2}'))
    return times

def find_moon_phases():
    f = moon_phases(eph)
    t, phase = find_discrete(t0, t1, f)
    labels = ['Новолуние', 'Первая четверть', 'Полнолуние', 'Последняя четверть']
    return [(t[i].utc_datetime(), labels[phase[i]]) for i in range(len(phase))]

def find_ingressions():
    entries = []
    observer = ephem.Observer()
    observer.date = DATE
    for planet in ['Mercury', 'Venus', 'Mars', 'Jupiter', 'Saturn', 'Sun', 'Moon']:
        p = getattr(ephem, planet)()
        p.compute(observer)
        sign = ephem.constellation(p)[1]
        entries.append((start_time.replace(hour=12), f'{planet} в знаке {sign}'))  # Упрощенно
    return entries

def get_conflict_zones():
    # Примитивная проверка падения/изгнания
    conflicts = {
        'Mars': ['Рак', 'Телец'],
        'Venus': ['Скорпион', 'Овен'],
        'Mercury': ['Рыбы'],
    }
    observer = ephem.Observer()
    observer.date = DATE
    result = []
    for planet, signs in conflicts.items():
        p = getattr(ephem, planet)()
        p.compute(observer)
        sign = ephem.constellation(p)[1]
        if sign in signs:
            result.append((start_time.replace(hour=14), f'{planet} в падении ({sign}) — напряжённое влияние'))
    return result

# -------------------- Главный блок --------------------

events = []

# Аспекты
aspects = find_aspects()
events.extend(aspects)

# Фазы луны
phases = find_moon_phases()
events.extend(phases)

# Ингрессии
ingressions = find_ingressions()
events.extend(ingressions)

# Конфликтные зоны
conflicts = get_conflict_zones()
events.extend(conflicts)

# Сортировка по времени
events.sort(key=lambda x: x[0])

# Вывод
for time, desc in events:
    time_str = time.strftime("%Y-%m-%d %H:%M")
    print(f"{time_str} — {desc}")


[#################################] 100% de421.bsp


2025-07-08 00:00 — Sun Тригон Venus
2025-07-08 00:00 — Mercury Оппозиция Uranus
2025-07-08 00:00 — Mercury Тригон Neptune
2025-07-08 00:00 — Venus Тригон Sun
2025-07-08 00:00 — Saturn Секстиль Uranus
2025-07-08 00:00 — Uranus Оппозиция Mercury
2025-07-08 00:00 — Uranus Секстиль Saturn
2025-07-08 00:00 — Neptune Тригон Mercury
2025-07-08 01:00 — Sun Тригон Venus
2025-07-08 01:00 — Mercury Оппозиция Uranus
2025-07-08 01:00 — Mercury Тригон Neptune
2025-07-08 01:00 — Venus Тригон Sun
2025-07-08 01:00 — Saturn Секстиль Uranus
2025-07-08 01:00 — Uranus Оппозиция Mercury
2025-07-08 01:00 — Uranus Секстиль Saturn
2025-07-08 01:00 — Neptune Тригон Mercury
2025-07-08 02:00 — Sun Тригон Venus
2025-07-08 02:00 — Mercury Оппозиция Uranus
2025-07-08 02:00 — Mercury Тригон Neptune
2025-07-08 02:00 — Venus Тригон Sun
2025-07-08 02:00 — Saturn Секстиль Uranus
2025-07-08 02:00 — Uranus Оппозиция Mercury
2025-07-08 02:00 — Uranus Секстиль Saturn
2025-07-08 02:00 — Neptune Тригон Mercury
2025-07-08 03:00

In [ ]:
import pandas as pd
from skyfield.api import load
from skyfield.api import N, E, wgs84
from skyfield.almanac import find_discrete, moon_phases
import ephem
from datetime import datetime, timedelta
import pytz

# Указанная дата анализа
DATE = '2025-05-30'
UTC = pytz.UTC
start_time = UTC.localize(datetime.strptime(DATE, "%Y-%m-%d"))
end_time = start_time + timedelta(days=1)

# Загрузка эфемерид
eph = load('de421.bsp')
ts = load.timescale()

# Время начала и конца дня
t0 = ts.utc(start_time.year, start_time.month, start_time.day)
t1 = ts.utc(end_time.year, end_time.month, end_time.day)

# Планеты
planets = {
    'Sun': eph['sun'],
    'Moon': eph['moon'],
    'Mercury': eph['mercury'],
    'Venus': eph['venus'],
    'Mars': eph['mars'],
    'Jupiter': eph['jupiter barycenter'],
    'Saturn': eph['saturn barycenter'],
    'Uranus': eph['uranus barycenter'],
    'Neptune': eph['neptune barycenter'],
    'Pluto': eph['pluto barycenter'],
}

aspect_angles = {
    0: 'Соединение',
    60: 'Секстиль',
    90: 'Квадрат',
    120: 'Тригон',
    180: 'Оппозиция',
}

# Функция для вычисления угла между планетами
def calc_angle(p1, p2, time):
    e1 = planets[p1].at(time).ecliptic_latlon()[1].degrees
    e2 = planets[p2].at(time).ecliptic_latlon()[1].degrees
    diff = abs(e1 - e2) % 360
    return min(diff, 360 - diff)

def find_aspects():
    times = []
    for hour in range(0, 24):
        t = ts.utc(start_time.year, start_time.month, start_time.day, hour)
        for p1 in planets:
            for p2 in planets:
                if p1 == p2:
                    continue
                angle = calc_angle(p1, p2, t)
                for target_angle, name in aspect_angles.items():
                    if abs(angle - target_angle) < 2.0:
                        times.append((t.utc_datetime(), f'{p1} {name} {p2}', t.utc_datetime(), f'{angle:.1f}°'))
    return times

def find_moon_phases():
    f = moon_phases(eph)
    t, phase = find_discrete(t0, t1, f)
    labels = ['Новолуние', 'Первая четверть', 'Полнолуние', 'Последняя четверть']
    return [(t[i].utc_datetime(), labels[phase[i]], t[i].utc_datetime(), '-') for i in range(len(phase))]

def find_ingressions():
    entries = []
    observer = ephem.Observer()
    observer.date = DATE
    for planet in ['Mercury', 'Venus', 'Mars', 'Jupiter', 'Saturn', 'Sun', 'Moon']:
        p = getattr(ephem, planet)()
        p.compute(observer)
        sign = ephem.constellation(p)[1]
        entries.append((start_time.replace(hour=12), f'{planet} ингресс в {sign}', start_time.replace(hour=12), '-'))
    return entries

def get_conflict_zones():
    conflicts = {
        'Mars': ['Рак', 'Телец'],
        'Venus': ['Скорпион', 'Овен'],
        'Mercury': ['Рыбы'],
    }
    observer = ephem.Observer()
    observer.date = DATE
    result = []
    for planet, signs in conflicts.items():
        p = getattr(ephem, planet)()
        p.compute(observer)
        sign = ephem.constellation(p)[1]
        if sign in signs:
            result.append((start_time.replace(hour=14), f'{planet} в падении ({sign})', start_time.replace(hour=14), '-'))
    return result

# 🎯 Автоматическая трактовка для трейдинга
def interpret_trading(event):
    if 'ингресс' in event:
        return "Изменение рыночных настроений, вероятен разворот или выход из боковика"
    elif 'Полнолуние' in event or 'Новолуние' in event:
        return "Кульминация или разворот тренда, повышенная волатильность"
    elif 'падении' in event or 'напряж' in event:
        return "Снижение силы импульсов, рост неуверенности"
    elif 'Соединение' in event:
        return "Момент активации — возможно начало нового движения"
    elif 'Оппозиция' in event:
        return "Конфликтная энергия — часто разворот"
    elif 'Квадрат' in event:
        return "Напряжение, возможны резкие движения"
    elif 'Тригон' in event or 'Секстиль' in event:
        return "Гармония — продолжение текущего движения"
    else:
        return "Нейтральное влияние"

# ---------- Главный блок ----------

events = []
events.extend(find_aspects())
events.extend(find_moon_phases())
events.extend(find_ingressions())
events.extend(get_conflict_zones())

# Удалим дубликаты по описанию события и времени
unique = list({(e[0], e[1]): e for e in events}.values())

# Создание DataFrame
df = pd.DataFrame(unique, columns=[
    'Дата', 'Событие', 'Дата активации', 'Градус'
])

# Добавим трактование
df['Трактовка (трейдинг)'] = df['Событие'].apply(interpret_trading)

# Фильтруем только лондонскую сессию (07:00 — 21:00 UTC)
df = df[df['Дата'].dt.hour.between(7, 21)]

# Сортировка по времени
df = df.sort_values(by='Дата')

# Вывод
df


,Дата,Событие,Дата активации,Градус,Трактовка (трейдинг)
70,2025-05-30 07:00:00+00:00,Sun Секстиль Venus,2025-05-30 07:00:00+00:00,61.8°,Гармония — продолжение текущего движения
71,2025-05-30 07:00:00+00:00,Moon Оппозиция Mercury,2025-05-30 07:00:00+00:00,178.6°,Конфликтная энергия — часто разворот
72,2025-05-30 07:00:00+00:00,Mercury Оппозиция Moon,2025-05-30 07:00:00+00:00,178.6°,Конфликтная энергия — часто разворот
73,2025-05-30 07:00:00+00:00,Venus Секстиль Sun,2025-05-30 07:00:00+00:00,61.8°,Гармония — продолжение текущего движения
74,2025-05-30 07:00:00+00:00,Mars Оппозиция Saturn,2025-05-30 07:00:00+00:00,179.0°,Конфликтная энергия — часто разворот
...,...,...,...,...,...
168,2025-05-30 21:00:00+00:00,Jupiter Квадрат Neptune,2025-05-30 21:00:00+00:00,91.3°,"Напряжение, возможны резкие движения"
169,2025-05-30 21:00:00+00:00,Saturn Оппозиция Mars,2025-05-30 21:00:00+00:00,178.7°,Конфликтная энергия — часто разворот
170,2025-05-30 21:00:00+00:00,Uranus Тригон Mars,2025-05-30 21:00:00+00:00,118.5°,Гармония — продолжение текущего движения
171,2025-05-30 21:00:00+00:00,Neptune Квадрат Jupiter,2025-05-30 21:00:00+00:00,91.3°,"Напряжение, возможны резкие движения"


In [ ]:
from skyfield.api import load
from skyfield.api import Topos
from datetime import datetime, timedelta
import pytz

# Настройки
SIGNIFICANT_PLANETS = {"Sun", "Moon", "Mercury", "Venus", "Mars", "Jupiter", "Saturn"}
GANN_DEGREES = [0, 7.5, 11.25, 22.5, 45, 60, 90, 135, 180, 225, 270, 315]
DEGREE_TOLERANCE = 2.5  # Погрешность
EVENT_DISTANCE_HOURS = 6  # фильтр уникальных событий

planets = load('de421.bsp')
ts = load.timescale()

# Горизонт времени
start = datetime(2025, 5, 30, 0, 0, tzinfo=pytz.UTC)
end = datetime(2025, 5, 30, 23, 59, tzinfo=pytz.UTC)
delta = timedelta(hours=1)

bodies = {
    "Sun": planets["sun"],
    "Moon": planets["moon"],
    "Mercury": planets["mercury"],
    "Venus": planets["venus"],
    "Mars": planets["mars"],
    "Jupiter": planets["jupiter barycenter"],
    "Saturn": planets["saturn barycenter"],
    "Uranus": planets["uranus barycenter"],
    "Neptune": planets["neptune barycenter"],
    "Pluto": planets["pluto barycenter"],
}

# Определение аспекта
def is_gann_angle(deg):
    return any(abs(deg - target) <= DEGREE_TOLERANCE for target in GANN_DEGREES)

def get_interpretation(deg):
    if abs(deg - 180) < 3 or abs(deg - 90) < 3 or abs(deg - 270) < 3:
        return "Конфликт — возможен разворот"
    elif abs(deg - 0) < 3 or abs(deg - 60) < 3 or abs(deg - 120) < 3:
        return "Гармония — продолжение движения"
    else:
        return "Особый аспект"

# Расчёт аспектов
last_seen = {}
results = {}

t = start
while t <= end:
    sf_time = ts.from_datetime(t)
    for p1 in SIGNIFICANT_PLANETS:
      for p2 in SIGNIFICANT_PLANETS:
          if p1 >= p2:
              continue  # Исключаем повторы

          lon1, lat1, dist1 = bodies[p1].at(sf_time).ecliptic_latlon()
          lon2, lat2, dist2 = bodies[p2].at(sf_time).ecliptic_latlon()

          angle = (lon1.degrees - lon2.degrees) % 360
          angle = round(angle, 1)

          if is_gann_angle(angle):
              key = f"{p1} аспект {p2}"
              last_time = last_seen.get(key)

              if not last_time or (t - last_time).total_seconds() > EVENT_DISTANCE_HOURS * 3600:
                  last_seen[key] = t
                  interpretation = get_interpretation(angle)
                  date_str = t.strftime("%Y-%m-%d")
                  time_str = t.strftime("%H:%M")
                  entry = f"- {time_str} — {p1} {p2} — {interpretation} ({angle}°)"
                  results.setdefault(date_str, []).append(entry)

    t += delta

# Вывод
for date, entries in sorted(results.items()):
    print(f"📆 {date}")
    for e in entries:
        print(e)


📆 2025-05-30
- 00:00 — Jupiter Venus — Гармония — продолжение движения (1.7°)
- 00:00 — Jupiter Saturn — Гармония — продолжение движения (2.0°)
- 00:00 — Mercury Sun — Гармония — продолжение движения (1.1°)
- 00:00 — Mercury Moon — Гармония — продолжение движения (2.4°)
- 00:00 — Moon Venus — Гармония — продолжение движения (1.9°)
- 00:00 — Moon Saturn — Гармония — продолжение движения (2.2°)
- 00:00 — Mars Sun — Гармония — продолжение движения (0.2°)
- 00:00 — Mars Moon — Гармония — продолжение движения (1.5°)
- 07:00 — Jupiter Venus — Гармония — продолжение движения (1.7°)
- 07:00 — Jupiter Saturn — Гармония — продолжение движения (2.0°)
- 07:00 — Mercury Sun — Гармония — продолжение движения (1.3°)
- 07:00 — Moon Venus — Гармония — продолжение движения (1.9°)
- 07:00 — Moon Saturn — Гармония — продолжение движения (2.2°)
- 07:00 — Mars Sun — Гармония — продолжение движения (0.2°)
- 07:00 — Mars Moon — Гармония — продолжение движения (1.5°)
- 13:00 — Mercury Saturn — Особый аспект (5

In [ ]:
from skyfield.api import load, utc
from datetime import datetime, timedelta

# Загрузка эфемерид и временной шкалы
eph = load('de421.bsp')
ts = load.timescale()

# Планеты и соответствия имен в efemerides
planet_names = {
    'Sun': 'sun',
    'Moon': 'moon',
    'Mercury': 'mercury',
    'Venus': 'venus',
    'Mars': 'mars',
    'Jupiter': 'jupiter barycenter',
    'Saturn': 'saturn barycenter',
    'Uranus': 'uranus barycenter',
    'Neptune': 'neptune barycenter',
    'Pluto': 'pluto barycenter'
}

bodies = {name: eph[real_name] for name, real_name in planet_names.items()}

# Важные градусы Ганна и допуск
gann_degrees = [7.5, 11.25, 22.5, 45, 60, 90, 180, 225, 270]
tolerance = 2.5

# Типы аспектов по градусам
aspect_types = {
    0: 'Соединение',
    7.5: 'Квинконс',
    11.25: 'Полуквадрат',
    22.5: 'Полуквадрат / Квинконс',
    30: 'Полусекстиль',
    45: 'Полуквадрат',
    60: 'Секстиль',
    90: 'Квадрат',
    120: 'Тригон',
    135: 'Полутораквадрат',
    180: 'Оппозиция',
    225: 'Квадрат',
    270: 'Тригон'
}

def get_aspect_name(angle):
    # Ищем ближайший аспект из aspect_types
    closest = min(aspect_types.keys(), key=lambda x: min(abs(angle - x), abs(angle - 360 - x)))
    diff = min(abs(angle - closest), abs(angle - 360 - closest))
    if diff <= tolerance:
        return aspect_types[closest]
    return None

# Временной диапазон
start = datetime(2025, 5, 30, tzinfo=utc)
end = datetime(2025, 5, 31, tzinfo=utc)
step_hours = 1

seen_aspects = set()
events_by_date = {}
previous_signs = {}

while start <= end:
    sf_time = ts.utc(start.year, start.month, start.day, start.hour, start.minute)

    # Аспекты
    for i, p1 in enumerate(planet_names):
        for p2 in list(planet_names.keys())[i+1:]:
            key = tuple(sorted([p1, p2]))
            if key in seen_aspects:
                continue
            lon1 = bodies[p1].at(sf_time).ecliptic_latlon()[0].degrees
            lon2 = bodies[p2].at(sf_time).ecliptic_latlon()[0].degrees
            angle = (lon1 - lon2) % 360
            # Проверяем соответствие аспекту
            aspect_name = get_aspect_name(angle)
            if aspect_name:
                record = f"{start.strftime('%H:%M')} — {p1} {aspect_name.lower()} {p2} — {'Гармония' if aspect_name in ['Секстиль', 'Тригон'] else 'Конфликт' if aspect_name in ['Квадрат', 'Оппозиция'] else 'Нейтрально'} ({round(angle, 1)}°)"
                events_by_date.setdefault(start.date(), []).append(record)
                seen_aspects.add(key)

    # Ингрессии
    for pname in planet_names:
        body = bodies[pname]
        lon = body.at(sf_time).ecliptic_latlon()[0].degrees
        current_sign = int(lon // 30)
        if pname in previous_signs and current_sign != previous_signs[pname]:
            zodiac = ['Овен', 'Телец', 'Близнецы', 'Рак', 'Лев', 'Дева',
                      'Весы', 'Скорпион', 'Стрелец', 'Козерог', 'Водолей', 'Рыбы'][current_sign]
            ingress = f"{start.strftime('%H:%M')} — {pname} входит в знак {zodiac} ({round(lon, 1)}°)"
            events_by_date.setdefault(start.date(), []).append(ingress)
        previous_signs[pname] = current_sign

    start += timedelta(hours=step_hours)

# Вывод в удобном формате
for date, events in sorted(events_by_date.items()):
    print(f"📆 {date}")
    for e in events:
        print(f"- {e}")
    print()


📆 2025-05-30
- 00:00 — Sun соединение Moon — Нейтрально (1.3°)
- 00:00 — Sun соединение Mercury — Нейтрально (358.9°)
- 00:00 — Sun соединение Mars — Нейтрально (359.8°)
- 00:00 — Sun соединение Jupiter — Нейтрально (1.5°)
- 00:00 — Sun соединение Uranus — Нейтрально (1.5°)
- 00:00 — Moon соединение Mercury — Нейтрально (357.6°)
- 00:00 — Moon соединение Venus — Нейтрально (1.9°)
- 00:00 — Moon соединение Mars — Нейтрально (358.5°)
- 00:00 — Moon соединение Jupiter — Нейтрально (0.2°)
- 00:00 — Moon соединение Saturn — Нейтрально (2.2°)
- 00:00 — Moon соединение Uranus — Нейтрально (0.2°)
- 00:00 — Moon соединение Neptune — Нейтрально (1.3°)
- 00:00 — Mercury соединение Mars — Нейтрально (0.9°)
- 00:00 — Mercury квинконс Pluto — Нейтрально (6.0°)
- 00:00 — Venus соединение Jupiter — Нейтрально (358.3°)
- 00:00 — Venus соединение Saturn — Нейтрально (0.3°)
- 00:00 — Venus соединение Uranus — Нейтрально (358.3°)
- 00:00 — Venus соединение Neptune — Нейтрально (359.4°)
- 00:00 — Venus сое

In [ ]:
from skyfield.api import load
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo

# Настройки
tolerance = 2.5  # Допуск градусов
step_minutes = 30  # Шаг времени для проверки аспектов

ny_tz = ZoneInfo('America/New_York')

def format_ny_time(dt_utc):
    return dt_utc.astimezone(ny_tz).strftime('%Y-%m-%d %H:%M %Z')

# Загрузка эфемерид и планет
eph = load('de421.bsp')
ts = load.timescale()

planet_names = {
    'Sun': 'sun',
    'Moon': 'moon',
    'Mercury': 'mercury',
    'Venus': 'venus',
    'Mars': 'mars',
    'Jupiter': 'jupiter barycenter',
    'Saturn': 'saturn barycenter',
    'Uranus': 'uranus barycenter',
    'Neptune': 'neptune barycenter',
    'Pluto': 'pluto barycenter'
}
bodies = {name: eph[real_name] for name, real_name in planet_names.items()}

# Типы аспектов и градусы
aspect_angles = [0, 22.5, 30, 45, 60, 90, 120, 135, 150, 180, 270]
aspect_types = {
    0: 'Соединение (Конфликт)',
    22.5: 'Полусекстиль (Нейтрально)',
    30: 'Полусекстиль (Нейтрально)',
    45: 'Квинконс (Нейтрально)',
    60: 'Секстиль (Гармония)',
    90: 'Квадрат (Конфликт)',
    120: 'Тригон (Гармония)',
    135: 'Квинтиль (Нейтрально)',
    150: 'Квинтиль (Нейтрально)',
    180: 'Оппозиция (Конфликт)',
    270: 'Квадрат (Конфликт)',
}

# Даты анализа
start = datetime(2025, 5, 30, tzinfo=timezone.utc)
end = datetime(2025, 6, 1, tzinfo=timezone.utc)

# Для хранения активных аспектов: ключ — пары планет, значение — dict с 'start', 'angle', 'type'
active_aspects = {}

# Для хранения событий ингрессий: планета -> предыдущий знак
previous_signs = {}

# Список всех событий для вывода
events = []

current_time = start
while current_time <= end:
    sf_time = ts.utc(current_time.year, current_time.month, current_time.day,
                     current_time.hour, current_time.minute)

    # Проверка аспектов
    for i, p1 in enumerate(planet_names):
        for p2 in list(planet_names.keys())[i+1:]:
            key = tuple(sorted([p1, p2]))
            body1 = bodies[p1]
            body2 = bodies[p2]

            lon1 = body1.at(sf_time).ecliptic_latlon()[0].degrees % 360
            lon2 = body2.at(sf_time).ecliptic_latlon()[0].degrees % 360
            angle = (lon1 - lon2) % 360

            # Ищем, есть ли аспект из списка по углу с допуском
            found_aspect = None
            for asp_angle in aspect_angles:
                diff = min(abs(angle - asp_angle), abs(360 - abs(angle - asp_angle)))
                if diff <= tolerance:
                    found_aspect = asp_angle
                    break

            if found_aspect is not None:
                asp_type = aspect_types[found_aspect]
                if key not in active_aspects:
                    # Начинается новый аспект
                    active_aspects[key] = {'start': current_time, 'angle': angle, 'type': asp_type}
                else:
                    # Аспект продолжается, можно обновить угол
                    active_aspects[key]['angle'] = angle
            else:
                # Аспект не обнаружен, проверяем, был ли он активен
                if key in active_aspects:
                    # Заканчиваем аспект
                    asp = active_aspects.pop(key)
                    events.append({
                        'type': 'Аспект',
                        'planets': key,
                        'aspect_type': asp['type'],
                        'start': asp['start'],
                        'end': current_time,
                        'angle': asp['angle']
                    })

    # Проверка ингрессий
    for pname in planet_names:
        body = bodies[pname]
        lon = body.at(sf_time).ecliptic_latlon()[0].degrees % 360
        current_sign = int(lon // 30)
        if pname in previous_signs:
            if current_sign != previous_signs[pname]:
                zodiac = ['Овен', 'Телец', 'Близнецы', 'Рак', 'Лев', 'Дева',
                          'Весы', 'Скорпион', 'Стрелец', 'Козерог', 'Водолей', 'Рыбы'][current_sign]
                events.append({
                    'type': 'Ингрессия',
                    'planet': pname,
                    'sign': zodiac,
                    'time': current_time
                })
        previous_signs[pname] = current_sign

    current_time += timedelta(minutes=step_minutes)

# По окончании цикла, закрываем все открытые аспекты (до конца периода)
for key, asp in active_aspects.items():
    events.append({
        'type': 'Аспект',
        'planets': key,
        'aspect_type': asp['type'],
        'start': asp['start'],
        'end': end,
        'angle': asp['angle']
    })

# Сортируем события по времени
def event_time(e):
    if e['type'] == 'Аспект':
        return e['start']
    else:
        return e['time']

events.sort(key=event_time)

# Выводим красиво с переводом в Нью-Йоркское время
for e in events:
    if e['type'] == 'Аспект':
        print(f"Аспект {e['planets'][0]} — {e['planets'][1]} ({e['aspect_type']}), "
              f"с {format_ny_time(e['start'])} по {format_ny_time(e['end'])}, "
              f"угол ~{round(e['angle'], 1)}°")
    elif e['type'] == 'Ингрессия':
        print(f"Ингрессия планеты {e['planet']} в знак {e['sign']} — {format_ny_time(e['time'])}")


Аспект Mercury — Moon (Соединение (Конфликт)), с 2025-05-29 20:00 EDT по 2025-05-30 00:00 EDT, угол ~357.5°
Аспект Mercury — Sun (Соединение (Конфликт)), с 2025-05-29 20:00 EDT по 2025-05-31 19:30 EDT, угол ~357.5°
Аспект Moon — Sun (Соединение (Конфликт)), с 2025-05-29 20:00 EDT по 2025-05-31 20:00 EDT, угол ~1.3°
Аспект Mars — Sun (Соединение (Конфликт)), с 2025-05-29 20:00 EDT по 2025-05-31 20:00 EDT, угол ~359.8°
Аспект Jupiter — Sun (Соединение (Конфликт)), с 2025-05-29 20:00 EDT по 2025-05-31 20:00 EDT, угол ~1.5°
Аспект Sun — Uranus (Соединение (Конфликт)), с 2025-05-29 20:00 EDT по 2025-05-31 20:00 EDT, угол ~1.5°
Аспект Moon — Venus (Соединение (Конфликт)), с 2025-05-29 20:00 EDT по 2025-05-31 20:00 EDT, угол ~2.1°
Аспект Mars — Moon (Соединение (Конфликт)), с 2025-05-29 20:00 EDT по 2025-05-31 20:00 EDT, угол ~358.5°
Аспект Jupiter — Moon (Соединение (Конфликт)), с 2025-05-29 20:00 EDT по 2025-05-31 20:00 EDT, угол ~0.2°
Аспект Moon — Saturn (Соединение (Конфликт)), с 2025-05